In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [4]:
import sys
sys.path.append('../src')

In [5]:
from data_transformation import get_better_spectra

In [6]:
from os import listdir

In [8]:
!ls

create_folds.py		low_error_model.keras  setup.py
data_generator.py	process_cas.py	       submission.csv
data_transformation.py	__pycache__	       ToFutils.py


In [11]:
a = get_new_spectra()

In [10]:
def get_new_spectra(dir = '../data/SSL_Csvs/'):
    '''
    Read in better peak info and trandform into lists of channels, intensities, names.
    '''
    channels = []
    intensities = []
    names = []
    times = []
    slopes = []
    offsets = []
    binsizes = []
    for csv in listdir(dir):
        if csv.split('.')[-1] == 'csv':
            data = pd.read_csv(dir + csv, names=list('abcdefgh'))
            if data.loc[9]['a'].split(':')[-1] == ' yes':
                channels.append(data.loc[21:]['a'].apply(float))
                intensities.append(data.loc[21:]['b'].apply(float))
                times.append(float(data.loc[6]['a'].split(':')[-1]))
                slopes.append(float(data.loc[11]['a'].split(':')[-1]))
                offsets.append( float(data.loc[12]['a'].split(':')[-1]))
                binsizes.append(float(data.loc[15]['a'].split(':')[-1]))
                names.append(csv[0:-3] + 'cas')
            else:
                channels.append(data.loc[21:]['a'].apply(float))
                intensities.append(data.loc[21:]['b'].apply(float))
                times.append(float(data.loc[6]['a'].split(':')[-1]))
                slopes.append('NaN')
                offsets.append('NaN')
                binsizes.append(float(data.loc[12]['a'].split(':')[-1]))
    
    return pd.DataFrame(list(zip(channels, intensities, names, times, slopes, offsets, binsizes)),
     columns=['precise_channels', 'precise_intensities', 'file_name', 'StartFlightTime', 'Mass/Time', 'MassOffset', 'SpecBinSize'])

In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   precise_channels     1912 non-null   object 
 1   precise_intensities  1912 non-null   object 
 2   file_name            1912 non-null   object 
 3   StartFlightTime      1912 non-null   float64
 4   Mass/Time            1912 non-null   float64
 5   MassOffset           1912 non-null   float64
 6   SpecBinSize          1912 non-null   float64
dtypes: float64(4), object(3)
memory usage: 104.7+ KB


In [ ]:
a.to_csv('../data/new_spectra.csv', index=False)

In [ ]:
from data_transformation import get_precise_peaks

In [ ]:
['peaks'] = get_precise_peaks(a, ['precise_channels', 'precise_intensities'])

In [ ]:
new = pd.read_csv('../data/new_spectra.csv')

In [ ]:
from ast import literal_eval
new['precise_channels']  = new['precise_channels'].apply(get_floats)
new['precise_intensities']  = new['precise_intensities'].apply(get_floats)
new['peaks']  = new['peaks'].apply(get_floats)

In [ ]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   precise_channels     1912 non-null   object 
 1   precise_intensities  1912 non-null   object 
 2   file_name            1912 non-null   object 
 3   StartFlightTime      1912 non-null   float64
 4   Mass/Time            1912 non-null   float64
 5   MassOffset           1912 non-null   float64
 6   SpecBinSize          1912 non-null   float64
 7   peaks                1912 non-null   object 
dtypes: float64(4), object(4)
memory usage: 119.6+ KB


In [ ]:
c = new['precise_channels'][0]

In [ ]:
a = map(str,c.split(','))

In [ ]:
float(new['precise_channels'][0].split('\n')[0].split()[1])

12729.6134

In [ ]:
def get_floats(string):
    floats = []
    for section in string.split('\n')[:-1]:
        v = section.split()
        if len(v) == 2:
            floats.append(float(v[1]))
    return floats

In [ ]:
new.to_csv('../data/new_spectra', index=False)